In [2]:
import boto3
import pandas as pd
import numpy as np
import math
from sagemaker import get_execution_role
import json
import s3fs
import gzip
import pickle

In [2]:
#Pull in game data
out_path = '../common/data/'

#blitz plays
blitz = pd.read_csv(out_path + 'blitz_plays.csv')
#player data
player = pd.read_pickle(out_path + 'player_play_rect.pkl')
#play data
play = pd.read_pickle(out_path + 'play_rect.pkl')
#play_df = pd.DataFrame(play, columns=['playId', 'gameId', 'play_type',
#                                      'yardlineNumber', 'playDesc', 
#                                      'dInBox','dRushers', 'sack', 'interception', 'qb_hit']) 

In [3]:
#Merge datasets against blitz plays
blitz_d = pd.merge(blitz, play, how='left', on=['playId', 'gameId'])
blitz_df = pd.merge(blitz_d, player, how='left', on=['playId', 'gameId'])
team_mapping = {'DB': 'defense', 'OL': 'offense', 'LB': 'defense', 'DL': 'defense', 
                'WR': 'offense', 'RB': 'offense', 'TE': 'offense', 'QB': 'offense'}
blitz_df['player_side'] = blitz_df.positionGroup.map(team_mapping)
blitz_df = blitz_df[(blitz_df['player_side'] == 'defense') | (blitz_df.position == 'QB')] #limit to defensive players & QBs

In [4]:
blitz['playId'].nunique

<bound method IndexOpsMixin.nunique of 0        424
1        698
2       1404
3       1885
4       2606
        ... 
3079    3620
3080    3700
3081    3722
3082    3768
3083    3807
Name: playId, Length: 3084, dtype: int64>

In [5]:
blitz_df['playId'].nunique #check merge

<bound method IndexOpsMixin.nunique of 2         424
3         424
4         424
5         424
6         424
         ... 
67841    3807
67842    3807
67843    3807
67844    3807
67845    3807
Name: playId, Length: 37009, dtype: int64>

In [6]:
pd.set_option('display.max_columns', None)
blitz_df.head(5)

,gameId,playId,home_team,away_team,homeScore,visitorScore,quarter,gameClock,startTime,endTime,snapTime_x,possessionTeam,playDesc,playType,playTypeCode,down,ydstogo,play_type,isGoalToGo,yardlineNumber,yardlineSide,absoluteYardlineNumber,isRedzonePlay,isScoring,isPenalty,oForm,dInBox,RB,TE,WR,DL,LB,DB,dRushers,shotgun,no_huddle,qb_dropback,qb_scramble,pass_location,air_yards,run_location,run_gap,ep,epa,wp,home_wp,wpa,first_down_rush,first_down_pass,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,interception,fumble_forced,tackled_for_loss,fumble_lost,qb_hit,rush_attempt,pass_attempt,sack,fumble,complete_pass,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,dir_x,snapPos_x,rectYardline,nflId,displayName,position,positionGroup,snapTime_y,teamType,play_endTime,coordinates,play_startIdx,play_endIdx,isOnField_atSnap,jerseyNumber,dir_y,snapPos_y,player_side
2,2019090500,424,CHI,GB,0,0,1.0,08:51,2019-09-06 00:36:16.750,2019-09-06 00:36:22.220,2019-09-06 00:36:16.750,GB,(Shotgun) A.Rodgers sacked at GB 5 for -7 yard...,play_type_sack,2.0,3.0,10,pass,False,12.0,GB,22.0,False,False,False,SHOTGUN,5.0,1.0,1.0,3.0,2.0,4.0,5.0,5.0,1,0,1.0,0,NaN,NaN,NaN,NaN,-2.00846,-1.484721,0.430832,0.569168,-0.046353,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,00-0033072,L.Floyd,00-0033072,L.Floyd,NaN,NaN,NaN,NaN,0.0,"[98.61, 26.05]",98.0,2495108,Prince Amukamara,CB,DB,2019-09-06 00:36:16.750,home,2019-09-06 00:36:22.220,"[[95.75, 12.82], [95.74, 12.81], [95.73, 12.8]...",228,313,True,20,0.0,"[98.61, 26.05]",defense
3,2019090500,424,CHI,GB,0,0,1.0,08:51,2019-09-06 00:36:16.750,2019-09-06 00:36:22.220,2019-09-06 00:36:16.750,GB,(Shotgun) A.Rodgers sacked at GB 5 for -7 yard...,play_type_sack,2.0,3.0,10,pass,False,12.0,GB,22.0,False,False,False,SHOTGUN,5.0,1.0,1.0,3.0,2.0,4.0,5.0,5.0,1,0,1.0,0,NaN,NaN,NaN,NaN,-2.00846,-1.484721,0.430832,0.569168,-0.046353,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,00-0033072,L.Floyd,00-0033072,L.Floyd,NaN,NaN,NaN,NaN,0.0,"[98.61, 26.05]",98.0,2495343,Buster Skrine,CB,DB,2019-09-06 00:36:16.750,home,2019-09-06 00:36:22.220,"[[95.1, 16.27], [95.1, 16.3], [95.1, 16.32], [...",228,313,True,24,0.0,"[98.61, 26.05]",defense
4,2019090500,424,CHI,GB,0,0,1.0,08:51,2019-09-06 00:36:16.750,2019-09-06 00:36:22.220,2019-09-06 00:36:16.750,GB,(Shotgun) A.Rodgers sacked at GB 5 for -7 yard...,play_type_sack,2.0,3.0,10,pass,False,12.0,GB,22.0,False,False,False,SHOTGUN,5.0,1.0,1.0,3.0,2.0,4.0,5.0,5.0,1,0,1.0,0,NaN,NaN,NaN,NaN,-2.00846,-1.484721,0.430832,0.569168,-0.046353,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,00-0033072,L.Floyd,00-0033072,L.Floyd,NaN,NaN,NaN,NaN,0.0,"[98.61, 26.05]",98.0,2506363,Aaron Rodgers,QB,QB,2019-09-06 00:36:16.750,away,2019-09-06 00:36:22.220,"[[103.23, 25.9], [103.23, 25.9], [103.22, 25.9...",228,313,True,12,0.0,"[98.61, 26.05]",offense
5,2019090500,424,CHI,GB,0,0,1.0,08:51,2019-09-06 00:36:16.750,2019-09-06 00:36:22.220,2019-09-06 00:36:16.750,GB,(Shotgun) A.Rodgers sacked at GB 5 for -7 yard...,play_type_sack,2.0,3.0,10,pass,False,12.0,GB,22.0,False,False,False,SHOTGUN,5.0,1.0,1.0,3.0,2.0,4.0,5.0,5.0,1,0,1.0,0,NaN,NaN,NaN,NaN,-2.00846,-1.484721,0.430832,0.569168,-0.046353,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,00-0033072,L.Floyd,00-0033072,L.Floyd,NaN,NaN,NaN,NaN,0.0,"[98.61, 26.05]",98.0,2532961,Danny Trevathan,ILB,LB,2019-09-06 00:36:16.750,home,2019-09-06 00:36:22.220,"[[94.61, 21.91], [94.59, 21.99], [94.56, 22.06...",228,313,True,59,0.0,"[98.61, 26.05]",defense
6,2019090500,424,CHI,GB,0,0,1.0,08:51,2019-09-06 00:36:16.750,2019-09-06 00:36:22.220,2019-09-06 00:36:16.750,GB,(Shotgun) A.Rodgers sacked at GB 5 for -7 yard...,play_type_sack,2.0,3.0,10,pass,False,12.0,GB,22.0,False,False,False,SHOTGUN,5.0,1.0,

# Develop Features

In [7]:
#pull xy coordinates for important points in the play - player, LOS
x=0
for i in blitz_df:
    x_snap = lambda x: x[20][0]
    y_snap = lambda x: x[20][1]
    LOS_x = lambda x: x[0]
    LOS_y = lambda x: x[1]
    x_1s = lambda x: x[30][0]
    y_1s = lambda x: x[30][1]
    x_2s = lambda x: x[40][0]
    y_2s = lambda x: x[40][1]
    x_3s = lambda x: x[50][0]
    y_3s = lambda x: x[50][1]
    x_end = lambda x: x[-11][0]
    y_end = lambda x: x[-11][1]
    
    blitz_df['x_snap'] = blitz_df['coordinates'].apply(x_snap)
    blitz_df['y_snap'] = blitz_df['coordinates'].apply(y_snap)
    blitz_df['LOS_x'] = blitz_df['snapPos_x'].apply(LOS_x)
    blitz_df['LOS_y'] = blitz_df['snapPos_x'].apply(LOS_y)
    blitz_df['x_1s'] = blitz_df['coordinates'].apply(x_1s)
    blitz_df['y_1s'] = blitz_df['coordinates'].apply(y_1s)
    blitz_df['x_2s'] = blitz_df['coordinates'].apply(x_2s)
    blitz_df['y_2s'] = blitz_df['coordinates'].apply(y_2s)
    blitz_df['x_3s'] = blitz_df['coordinates'].apply(x_3s)
    blitz_df['y_3s'] = blitz_df['coordinates'].apply(y_3s)
    blitz_df['x_end'] = blitz_df['coordinates'].apply(x_end)
    blitz_df['y_end'] = blitz_df['coordinates'].apply(y_end)
    
    
print(blitz_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37009 entries, 2 to 67845
Columns: 102 entries, gameId to y_end
dtypes: bool(2), float64(52), int64(12), object(36)
memory usage: 28.6+ MB
None


In [53]:
#Assess pass rushers
blitz_df['x_player_pos_v_LOS'] = blitz_df['x_snap'] - blitz_df['LOS_x']
blitz_df['y_player_pos_v_LOS'] = blitz_df['y_snap'] - blitz_df['LOS_y']
blitz_df['x_1s_toLOS'] = blitz_df['x_1s'] - blitz_df['LOS_x']
blitz_df['y_1s_toLOS'] = blitz_df['y_1s'] - blitz_df['LOS_y']
blitz_df['x_2s_toLOS'] = blitz_df['x_2s'] - blitz_df['LOS_x']
blitz_df['y_2s_toLOS'] = blitz_df['y_2s'] - blitz_df['LOS_y']
blitz_df['x_3s_toLOS'] = blitz_df['x_3s'] - blitz_df['LOS_x']
blitz_df['y_3s_toLOS'] = blitz_df['y_3s'] - blitz_df['LOS_y']
#Define pass rushers
blitz_df['crossLOS']  = (blitz_df['x_2s_toLOS'] > -0.2)
blitz_df['crossLOS2']  = (blitz_df['x_1s_toLOS'] > -0.2)
blitz_df['y_bounds'] = ((abs(blitz_df['y_1s'] - blitz_df['LOS_y']) < 7))
blitz_df['y_bounds2'] = ((abs(blitz_df['y_3s'] - blitz_df['LOS_y']) < 6))
blitz_df['rusher'] = (((blitz_df['crossLOS2'] == True) | (blitz_df['crossLOS'] == True)) & ((blitz_df['y_bounds'] == True) | (blitz_df['y_bounds2'] == True))) 

In [54]:
#Separate out QBs & load back in
qb = blitz_df[blitz_df.position == 'QB']
qb['qb_xend'] = qb['x_end'] #QB at end of play
qb['qb_yend'] = qb['y_end'] #QB at end of play
qb = pd.DataFrame(qb, columns=['playId', 'gameId', 'qb_xend', 'qb_yend'])
blitz_df = pd.merge(blitz_df, qb, how='left', on=['playId', 'gameId'])

In [55]:
blitz_df['playId'].nunique #check merge

<bound method IndexOpsMixin.nunique of 0         424
1         424
2         424
3         424
4         424
         ... 
33920    3807
33921    3807
33922    3807
33923    3807
33924    3807
Name: playId, Length: 33925, dtype: int64>

In [56]:
#Create features 
blitz_df['vertPlay'] = abs(blitz_df['x_end'] - blitz_df['x_snap'])
blitz_df['horizPlay'] = abs(blitz_df['y_end'] - blitz_df['y_snap'])
blitz_df['vert1s'] = abs(blitz_df['x_1s'] - blitz_df['x_snap'])
blitz_df['vert3s'] = abs(blitz_df['x_3s'] - blitz_df['x_snap'])
blitz_df['horiz3s'] = abs(blitz_df['y_3s'] - blitz_df['y_snap'])
blitz_df['xfromQB'] = abs(blitz_df['qb_xend'] - blitz_df['x_end'])
blitz_df['yfromQB'] = abs(blitz_df['qb_yend'] - blitz_df['y_end'])
blitz_df['QBHurry'] = blitz_df['xfromQB'] < 1
blitz_df['vertSpeed_1s'] = blitz_df['vert1s']/3
blitz_df['vertSpeed_3s'] = blitz_df['vert3s']/3
#blitz_df['magnitude'] = math.sqrt((blitz_df['y_end']-blitz_df['y_snap'])**2 + (blitz_df['x_end']-blitz_df['x_snap'])**2)
#blitz_df['angle'] = math.atan2((blitz_df['y_end']-blitz_df['y_snap']),(blitz_df['x_end']-blitz_df['x_snap']))*(180/(math.pi))

KeyError: 'qb_xend'

# Analyze Features

In [58]:
#Pull in game data
out_path = '../common/data/'
blitz_df = pd.read_pickle(out_path + 'blitz_player_play.pkl')
blitz_df = blitz_df[(blitz_df['player_side'] == 'defense')] #limit to defensive players
blitz_df['unique_player_key'] = blitz_df['displayName'].astype(str) + "_"  + blitz_df['jerseyNumber'].astype(str) #add unique player ID
#blitz_df.head(5)

In [59]:
#Add in rusher methodology
blitz_df['crossLOS']  = (blitz_df['x_2s_toLOS'] > -0.2)
blitz_df['crossLOS2']  = (blitz_df['x_1s_toLOS'] > -0.2)
blitz_df['y_bounds'] = ((abs(blitz_df['y_1s'] - blitz_df['y_snap_ball']) < 7))
blitz_df['y_bounds2'] = ((abs(blitz_df['y_3s'] - blitz_df['y_snap_ball']) < 6))
blitz_df['rusher'] = ((blitz_df['player_side'] == 'defense') & (((blitz_df['crossLOS2'] == True) | (blitz_df['crossLOS'] == True)) & ((blitz_df['y_bounds'] == True) | (blitz_df['y_bounds2'] == True)))) 

In [17]:
#blitz_df.to_pickle(out_path + 'blitz_player_play.pkl')

In [60]:
#Create features 
blitz_df['vertPlay'] = abs(blitz_df['x_end'] - blitz_df['x_snap'])
blitz_df['horizPlay'] = abs(blitz_df['y_end'] - blitz_df['y_snap'])
blitz_df['vert1s'] = abs(blitz_df['x_1s'] - blitz_df['x_snap'])
blitz_df['vert3s'] = abs(blitz_df['x_3s'] - blitz_df['x_snap'])
blitz_df['horiz3s'] = abs(blitz_df['y_3s'] - blitz_df['y_snap'])
blitz_df['xfromQB'] = abs(blitz_df['qb_xend'] - blitz_df['x_end'])
blitz_df['yfromQB'] = abs(blitz_df['qb_yend'] - blitz_df['y_end'])
blitz_df['QBHurry'] = blitz_df['xfromQB'] < 1
blitz_df['vertSpeed_1s'] = blitz_df['vert1s']/3
blitz_df['vertSpeed_3s'] = blitz_df['vert3s']/3

In [61]:
#Define success metric
blitz_df['success'] = (blitz_df['sack'] == 1) | (blitz_df['interception'] == 1)

In [ ]:
#################################################################
###              Create rusher stats dataset                  ###
#################################################################

In [62]:
rusher = blitz_df[(blitz_df['rusher'] == True)]

In [63]:
#Who consistently gets closest to the QB
xQB = rusher.groupby(['displayName', 'position', 'unique_player_key'])['xfromQB'].mean().reset_index(name='avg_xfromQB')
xQB.sort_values(by='avg_xfromQB', ascending=True)

,displayName,position,unique_player_key,avg_xfromQB
646,Shaquill Griffin,CB,Shaquill Griffin_26,0.070
511,Marquise Blair,DB,Marquise Blair_27,0.070
365,Jimmy Smith,CB,Jimmy Smith_22,0.170
176,D.J. Reed,DB,D.J. Reed_32,0.170
569,Olivier Vernon,DE,Olivier Vernon_54,0.200
...,...,...,...,...
58,Avonte Maddox,CB,Avonte Maddox_29,20.720
487,Malik Hooker,FS,Malik Hooker_29,20.815
391,Josey Jewell,LB,Josey Jewell_47,21.010
572,Orlando Scandrick,CB,Orlando Scandrick_38,21.975


In [64]:
#Who is getting closest to the QB the most
QBh = rusher[rusher['QBHurry'] == True].groupby(['displayName', 'position', 'unique_player_key'])['QBHurry'].count().reset_index(name='QBHurry_cnt')
QBh.sort_values(by='QBHurry_cnt', ascending=False)

,displayName,position,unique_player_key,QBHurry_cnt
358,Markus Golden,OLB,Markus Golden_44,24
319,Kyle Van Noy,OLB,Kyle Van Noy_53,23
534,Whitney Mercilus,OLB,Whitney Mercilus_59,23
394,Ndamukong Suh,DT,Ndamukong Suh_93,21
3,Aaron Donald,DT,Aaron Donald_99,21
...,...,...,...,...
203,Ha Ha Clinton-Dix,FS,Ha Ha Clinton-Dix_21,1
201,Grant Haley,CB,Grant Haley_34,1
418,Preston Brown,MLB,Preston Brown_52,1
197,George Odum,FS,George Odum_30,1


In [65]:
#Who is the fastest in 1 sec
s_1 = rusher.groupby(['displayName', 'position', 'unique_player_key'])['vertSpeed_1s'].max().reset_index(name='max_vertSpeed_1s')
s_1.sort_values(by='max_vertSpeed_1s', ascending=False)

,displayName,position,unique_player_key,max_vertSpeed_1s
200,Darnell Savage,DB,Darnell Savage_26,2.116667
257,Duron Harmon,FS,Duron Harmon_21,2.053333
412,Justin Reid,SS,Justin Reid_20,2.020000
496,Marcus Gilchrist,FS,Marcus Gilchrist_29,2.003333
37,Anthony Harris,FS,Anthony Harris_41,1.976667
...,...,...,...,...
326,Jalen Mills,CB,Jalen Mills_31,0.216667
423,Keanu Neal,FS,Keanu Neal_22,0.206667
757,Xavien Howard,CB,Xavien Howard_25,0.190000
669,Stephone Anthony,MLB,Stephone Anthony_58,0.146667


In [66]:
#Who is the fastest in 3 sec - rusher
s_3 = rusher.groupby(['displayName', 'position', 'unique_player_key'])['vertSpeed_3s'].max().reset_index(name='max_vertSpeed_3s')
s_3.sort_values(by='max_vertSpeed_3s', ascending=False)

,displayName,position,unique_player_key,max_vertSpeed_3s
358,Jerome Baker,LB,Jerome Baker_55,6.900000
706,Tre Boston,FS,Tre Boston_33,6.496667
496,Marcus Gilchrist,FS,Marcus Gilchrist_29,6.420000
431,Kenny Vaccaro,SS,Kenny Vaccaro_24,6.130000
495,Marcus Epps,SS,Marcus Epps_39,6.076667
...,...,...,...,...
251,Dontavius Russell,DT,Dontavius Russell_98,0.266667
93,Brian Burns,DL,Brian Burns_53,0.260000
759,Xavier Woods,FS,Xavier Woods_25,0.210000
136,Chandon Sullivan,CB,Chandon Sullivan_39,0.206667


In [67]:
#Who is consistently fast in 1 sec - rushers
sa_1 = rusher.groupby(['displayName', 'position', 'unique_player_key'])['vertSpeed_1s'].mean().reset_index(name='avg_vertSpeed_1s')
sa_1.sort_values(by='avg_vertSpeed_1s', ascending=False)

,displayName,position,unique_player_key,avg_vertSpeed_1s
496,Marcus Gilchrist,FS,Marcus Gilchrist_29,2.003333
706,Tre Boston,FS,Tre Boston_33,1.840000
274,Eric Weddle,FS,Eric Weddle_32,1.771667
495,Marcus Epps,SS,Marcus Epps_39,1.750000
539,Montae Nicholson,FS,Montae Nicholson_35,1.604444
...,...,...,...,...
525,Michael Davis,CB,Michael Davis_43,0.208333
423,Keanu Neal,FS,Keanu Neal_22,0.206667
757,Xavien Howard,CB,Xavien Howard_25,0.190000
669,Stephone Anthony,MLB,Stephone Anthony_58,0.146667


In [68]:
#Who is consistently fast in 3 sec - rushers
sa_3 = rusher.groupby(['displayName', 'position', 'unique_player_key'])['vertSpeed_3s'].mean().reset_index(name='avg_vertSpeed_3s')
sa_3.sort_values(by='avg_vertSpeed_3s', ascending=False)

,displayName,position,unique_player_key,avg_vertSpeed_3s
706,Tre Boston,FS,Tre Boston_33,6.496667
496,Marcus Gilchrist,FS,Marcus Gilchrist_29,6.420000
495,Marcus Epps,SS,Marcus Epps_39,6.076667
207,David Long,CB,David Long_25,5.006667
532,Mike Hughes,CB,Mike Hughes_21,4.880000
...,...,...,...,...
251,Dontavius Russell,DT,Dontavius Russell_98,0.266667
93,Brian Burns,DL,Brian Burns_53,0.260000
759,Xavier Woods,FS,Xavier Woods_25,0.210000
136,Chandon Sullivan,CB,Chandon Sullivan_39,0.206667


In [69]:
#Who is rushing the most
vol = rusher.groupby(['displayName', 'position', 'unique_player_key'])['playId'].count().reset_index(name='rush_cnt')
vol.sort_values(by='rush_cnt', ascending=False)

,displayName,position,unique_player_key,rush_cnt
645,Shaquil Barrett,OLB,Shaquil Barrett_58,136
118,Cameron Jordan,DE,Cameron Jordan_94,135
555,Ndamukong Suh,DT,Ndamukong Suh_93,130
742,Vita Vea,DT,Vita Vea_50,125
749,Whitney Mercilus,OLB,Whitney Mercilus_59,119
...,...,...,...,...
611,Ricardo Allen,SS,Ricardo Allen_37,1
156,Clayton Geathers,FS,Clayton Geathers_26,1
605,Raven Greene,FS,Raven Greene_24,1
435,Kerry Wynn,DE,Kerry Wynn_72,1


In [70]:
#Who is causes the most success
suc = rusher[rusher['success'] == True].groupby(['displayName', 'position', 'unique_player_key'])['success'].count().reset_index(name='success_cnt')
suc.sort_values(by='success_cnt', ascending=False)

,displayName,position,unique_player_key,success_cnt
328,Matt Judon,OLB,Matt Judon_99,15
74,Cameron Jordan,DE,Cameron Jordan_94,13
240,Jihad Ward,DT,Jihad Ward_53,13
36,Arik Armstead,DE,Arik Armstead_91,12
359,Nick Bosa,DE,Nick Bosa_97,12
...,...,...,...,...
235,Jermaine Carter,LB,Jermaine Carter_56,1
241,Jimmy Smith,CB,Jimmy Smith_22,1
245,John Atkins,NT,John Atkins_99,1
249,Johnathan Hankins,DT,Johnathan Hankins_90,1


In [87]:
#Export rusher dataset
rusher_exp = rusher['unique_player_key'].drop_duplicates()
#rusher_exp.shape
rusher_exp = pd.merge(rusher_exp, xQB[['unique_player_key','position', 'avg_xfromQB']], how='left', on=['unique_player_key'])
rusher_exp = pd.merge(rusher_exp, QBh[['unique_player_key', 'QBHurry_cnt']], how='left', on=['unique_player_key'])
rusher_exp = pd.merge(rusher_exp, s_1[['unique_player_key', 'max_vertSpeed_1s']], how='left', on=['unique_player_key'])
rusher_exp = pd.merge(rusher_exp, s_3[['unique_player_key', 'max_vertSpeed_3s']], how='left', on=['unique_player_key'])
rusher_exp = pd.merge(rusher_exp, sa_1[['unique_player_key', 'avg_vertSpeed_1s']], how='left', on=['unique_player_key'])
rusher_exp = pd.merge(rusher_exp, sa_3[['unique_player_key', 'avg_vertSpeed_3s']], how='left', on=['unique_player_key'])
rusher_exp = pd.merge(rusher_exp, vol[['unique_player_key', 'rush_cnt']], how='left', on=['unique_player_key'])
rusher_exp = pd.merge(rusher_exp, suc[['unique_player_key', 'success_cnt']], how='left', on=['unique_player_key'])
#rusher_exp.head(10)
#rusher_exp.shape
rusher_exp.to_csv(out_path + 'rusher_metrics.csv')

In [80]:
#rusher_exp[rusher_exp['unique_player_key'] == 'Khalil Mack_52']